In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import pandas as pd
pd.set_option('display.max_columns',40)

In [9]:
from google.colab import drive
import os

# Mount Google Drive
#drive.mount('/content/drive')

# Path to your data folder in Google Drive
data_folder = '/content/drive/MyDrive/Real_Trader_EDA_R1/60_Minute_Data/60_minute_Raw_Data'

# List files in the specified folder to confirm access
folder_path = os.path.join(data_folder)  # Update 'path_to_your_data_folder' with your actual folder path
file_list = os.listdir(folder_path)
file_list

['path',
 'ES 09-24_Minute_60.csv',
 'NQ 09-24_Minute_60.csv',
 'ZB 09-24_Minute_60.csv']

In [10]:
import os
import pandas as pd

def generate_trading_features(df):
    # Initialize the new columns with zeros
    df['Entry'] = 0
    df['Target_b'] = 0
    df['Target_s'] = 0
    df['Defence_buyer'] = 0
    df['Defence_sale'] = 0
    df['Stop_buyer'] = 0
    df['Stop_sale'] = 0

    # Iterate over each row to calculate the features
    for index, row in df.iterrows():
        buyers_entry = 0
        sellers_entry = 0
        target_b = 0
        target_s = 0
        defence_buyer = 0
        defence_sale = 0
        stop_buyer = 0
        stop_sale = 0

        if pd.notna(row['BuyersInControlPrice']) and row['BuyersInControlPrice'] != 0:
            buyers_entry = row['BuyersInControlPrice']
            target_b = buyers_entry + 2
            defence_buyer = buyers_entry - 1  # Default defense for buyer
            stop_buyer = buyers_entry - 2     # Default stop for buyer

        if pd.notna(row['SellersInControlPrice']) and row['SellersInControlPrice'] != 0:
            sellers_entry = row['SellersInControlPrice']
            target_s = sellers_entry - 2
            defence_sale = sellers_entry + 1  # Modified defense for seller
            stop_sale = sellers_entry + 2     # Modified stop for seller

        # Use the non-zero value as the entry
        entry = buyers_entry if buyers_entry != 0 else sellers_entry

        df.at[index, 'Entry'] = entry

        # Determine the defence and stop values
        df.at[index, 'Defence'] = defence_buyer if buyers_entry != 0 else defence_sale
        df.at[index, 'Stop'] = stop_buyer if buyers_entry != 0 else stop_sale

        # Determine the target values
        df.at[index, 'Target_b'] = target_b if buyers_entry != 0 else 0
        df.at[index, 'Target_s'] = target_s if sellers_entry != 0 else 0

    # Create a new column for combined target
    df["Target"] = df["Target_b"] + df["Target_s"]

    # Drop the individual target columns
    df.drop(columns=["Defence_buyer", "Defence_sale", "Stop_buyer", "Stop_sale", "Target_b", "Target_s"], axis=1, inplace=True)

    return df

def process_files_in_directory(input_directory, output_directory):
    # Ensure the output directory exists
    os.makedirs(output_directory, exist_ok=True)

    # Get all CSV files in the input directory
    files = [f for f in os.listdir(input_directory) if f.endswith('.csv')]

    for file in files:
        file_path = os.path.join(input_directory, file)

        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)

        # Convert 'Time Stamp' to datetime
        if 'Time Stamp' in df.columns:
            df['Time Stamp'] = pd.to_datetime(df['Time Stamp'], errors='coerce')

        # Drop the "Unnamed: 0" column if it exists
        if 'Unnamed: 0' in df.columns:
            df.drop(columns=['Unnamed: 0'], inplace=True)

        # Drop the specified columns if they exist
        columns_to_drop = ['Entry', 'Defence', 'Stop', 'Target']
        df.drop(columns=[col for col in columns_to_drop if col in df.columns], inplace=True)

        # Apply the trading feature generation function
        df = generate_trading_features(df)

        # Save the modified DataFrame to the output directory with prefix 'ESDT_'
        output_file_name = f"ESDT_{file}"
        output_file_path = os.path.join(output_directory, output_file_name)
        df.to_csv(output_file_path, index=False)

        print(f"Processed and saved {output_file_name} to {output_directory}")


In [11]:
# Example usage
input_directory = '/content/drive/MyDrive/Real_Trader_EDA_R1/60_Minute_Data/60_minute_Raw_Data'
output_directory = '/content/drive/MyDrive/Real_Trader_EDA_R1/60_Minute_Data/Calculate_60_Minute_Data/Distance_Based_ESDT_Data'
process_files_in_directory(input_directory, output_directory)


<ipython-input-10-59eb8e62079b>:40: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '3212.25' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[index, 'Entry'] = entry
<ipython-input-10-59eb8e62079b>:47: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '3214.25' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[index, 'Target_b'] = target_b if buyers_entry != 0 else 0
<ipython-input-10-59eb8e62079b>:48: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '3216.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[index, 'Target_s'] = target_s if sellers_entry != 0 else 0


Processed and saved ESDT_ES 09-24_Minute_60.csv to /content/drive/MyDrive/Real_Trader_EDA_R1/60_Minute_Data/Calculate_60_Minute_Data/Distance_Based_ESDT_Data


<ipython-input-10-59eb8e62079b>:40: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '9150.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[index, 'Entry'] = entry
<ipython-input-10-59eb8e62079b>:48: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '9148.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[index, 'Target_s'] = target_s if sellers_entry != 0 else 0
<ipython-input-10-59eb8e62079b>:47: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '9152.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[index, 'Target_b'] = target_b if buyers_entry != 0 else 0


Processed and saved ESDT_NQ 09-24_Minute_60.csv to /content/drive/MyDrive/Real_Trader_EDA_R1/60_Minute_Data/Calculate_60_Minute_Data/Distance_Based_ESDT_Data


<ipython-input-10-59eb8e62079b>:40: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '146.15625' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[index, 'Entry'] = entry
<ipython-input-10-59eb8e62079b>:48: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '144.15625' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[index, 'Target_s'] = target_s if sellers_entry != 0 else 0
<ipython-input-10-59eb8e62079b>:47: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '147.71875' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[index, 'Target_b'] = target_b if buyers_entry != 0 else 0


Processed and saved ESDT_ZB 09-24_Minute_60.csv to /content/drive/MyDrive/Real_Trader_EDA_R1/60_Minute_Data/Calculate_60_Minute_Data/Distance_Based_ESDT_Data
